In [2]:
import os
import sys
import math
import shutil
import tarfile
from pathlib import Path

from huggingface_hub import snapshot_download

/home/panda/miniconda3/envs/pytorch/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
data_path = snapshot_download(repo_id="neuroback/DataBack", repo_type="dataset", cache_dir="./full_data")
data_path = Path(data_path)

Fetching 10 files: 100%|██████████| 10/10 [00:00<00:00, 206615.96it/s]


In [ ]:
#  Unzip dataset files

DATA_CACHE_PATH = Path("./full_data")
TARGET_PATH = Path("./processed_data")

DATA_CACHE_PATH.mkdir(parents=True, exist_ok=True)
TARGET_PATH.mkdir(parents=True, exist_ok=True)

datasets = ["original", "dual"]

for d_name in datasets:
    search_dir = data_path / d_name
    targz_files = list(search_dir.glob("*.tar.gz"))

    for targz in targz_files:
        print(f"Sorting {targz.name}...")
 
        try:
            tarfile.open(targz, "r:gz").extractall(path=TARGET_PATH)
        except tarfile.ReadError:
            print(f"Could not extract {targz.name}", file=sys.stderr)
            continue


Sorting bb_pt.tar.gz...
Sorting cnf_pt.tar.gz...
Sorting bb_ft.tar.gz...
Sorting cnf_ft.tar.gz...
Sorting d_cnf_pt.tar.gz...
Sorting d_bb_pt.tar.gz...
Sorting d_cnf_ft.tar.gz...
Sorting d_bb_ft.tar.gz...


In [ ]:
# Split dataset into train, validation and test and create symlinks

TARGET_PATH = Path("./processed_data")
DATASET_PATH = Path("./data")

TRAIN_RATIO = 0.7
VALIDATION_RATIO = 0.2
TEST_RATIO = 0.1

if DATASET_PATH.exists():
    shutil.rmtree(DATASET_PATH)

DATASET_PATH.mkdir(parents=True, exist_ok=True)

features_source_cnf = TARGET_PATH / "cnf_pt"
y_source_bb = TARGET_PATH / "bb_pt"

features_source_d_cnf = TARGET_PATH / "d_cnf_pt"
y_source_d_bb = TARGET_PATH / "d_bb_pt"

sources = [features_source_cnf, y_source_bb, features_source_d_cnf, y_source_d_bb]
sources_path = ["cnf", "backbone", "cnf", "backbone"]

for source, s_path in zip(sources, sources_path):
    items = [item for item in source.iterdir() if not item.is_dir()]
    items.sort() # Se podria hacer random
    n_total = len(items)

    train_count = math.floor(n_total * TRAIN_RATIO)
    val_count = math.floor(n_total * VALIDATION_RATIO)

    train = items[:train_count]
    val = items[train_count:train_count + val_count]
    test = items[train_count + val_count:]

    dataset_items = [train, val, test]
    dataset_paths = ["pretrain", "validation", "test"]

    for split_items, split_path in zip(dataset_items, dataset_paths):
        sym_path = DATASET_PATH / s_path / split_path
        sym_path.mkdir(parents=True, exist_ok=True)

        for item in split_items:
            sym_path = DATASET_PATH / s_path / split_path / item.name
            item_path = item.resolve()

            os.symlink(item_path, sym_path)


In [ ]:
# Create finetune dataset symlinks

TARGET_PATH = Path("./processed_data")
DATASET_PATH = Path("./data")

#if DATASET_PATH.exists():
#    shutil.rmtree(DATASET_PATH)

#DATASET_PATH.mkdir(parents=True, exist_ok=True)

cnf_ft = TARGET_PATH / "cnf_ft"
bb_ft = TARGET_PATH / "bb_ft"

d_cnf_ft = TARGET_PATH / "d_cnf_ft"
d_bb_ft = TARGET_PATH / "d_bb_ft"

sources = [cnf_ft, bb_ft, d_cnf_ft, d_bb_ft]
sources_path = ["cnf", "backbone", "cnf", "backbone"]

for source, s_path in zip(sources, sources_path):
    items = [item for item in source.iterdir() if not item.is_dir()]
    items.sort() # Se podria hacer random

    sym_path = DATASET_PATH / s_path / "finetune"
    sym_path.mkdir(parents=True, exist_ok=True)

    for item in items:
        sym_path = DATASET_PATH / s_path / "finetune" / item.name
        item_path = item.resolve()

        os.symlink(item_path, sym_path)

In [22]:
root_dir = "./processed_data"
root_dir = Path("./processed_data/cnf_pt")
cnf_dir_list = [p for p in root_dir.iterdir() if p.is_file()]

print(root_dir)

idx = 100
print(cnf_dir_list[idx])

print(os.path.getsize(cnf_dir_list[idx]) / 1024)
print(cnf_dir_list[idx].stat().st_size / 1024)

processed_data/cnf_pt
processed_data/cnf_pt/CBS_k3_n100_m423_b10_535.cnf.xz
2.078125
2.078125


In [ ]:
import torch
import torch.nn as nn
from torch_geometric.loader import DataLoader
import os
import sys
import time
import argparse
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# --- IMPORTACIÓN DE LA NUEVA RED ---
from mamba_model import NeuroBackMamba
from data import MyOwnDataset, SortedBucketSampler

# Configuración de argumentos
parser = argparse.ArgumentParser()
parser.add_argument('--mode', type=str, required=True, choices=['pretrain', 'finetune'])
parser.add_argument('--batch_size', type=int, default=8)
parser.add_argument('--lr', type=float, default=1e-4)
parser.add_argument('--epochs', type=int, default=40)
parser.add_argument('--hidden_dim', type=int, default=64)
parser.add_argument('--layers', type=int, default=12)
args = parser.parse_args()

# Configuración de Rutas
BASE_LOG = f"./log/mamba_pretrain"
BASE_MODEL = f"./models/mamba_pretrain"
os.makedirs(BASE_LOG, exist_ok=True)
os.makedirs(BASE_MODEL, exist_ok=True)

# Configuración de Dispositivo y Semilla
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(77)

# --- CARGA DE DATOS ---
dataset_path = f"./data/pt/{args.mode}" 
dataset_train = MyOwnDataset(root=dataset_path)
dataset_vld = MyOwnDataset(root='./data/pt/validation')

batch_size_vld = 2

# sampler_train = SortedBucketSampler(dataset_train, args.batch_size, shuffle=True)
# sampler_vld = SortedBucketSampler(dataset_train, batch_size_vld, shuffle=False)
#
# train_loader = DataLoader(dataset_train, batch_size=args.batch_size, sampler=sampler_train, num_workers=12, pin_memory=False)
# vld_loader = DataLoader(dataset_vld, batch_size_vld, sampler=sampler_vld, num_workers=4, pin_memory=False)

train_loader = DataLoader(dataset_train, batch_size=args.batch_size, shuffle=True, num_workers=12, pin_memory=True)
vld_loader = DataLoader(dataset_vld, batch_size_vld, shuffle=False, num_workers=4, pin_memory=True)

# --- INICIALIZACIÓN DEL MODELO ---
input_dim = dataset_train.num_node_features 
print(input_dim)


/home/panda/miniconda3/envs/pytorch/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'mamba_ssm'